In [1]:
#--1--

# =========================================================
#  SETUP Imports & Input
#     - All imports in a single block (course style)
#     - Load the CSV exported from Stage 3 (Data Cleansing)
#     - Quick integrity checks (df.shape, df.info(), df.head())
# =========================================================

# ---------- 1.1 Imports (single place) ----------
import pandas as pd
import numpy as np
from pathlib import Path
import warnings

import pickle


from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder   
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif




warnings.filterwarnings("ignore")  # keep the notebook tidy, as in class

# ---------- 1.2 Define input path (Windows -> Pathlib) ----------
# NOTE: Replace with your exact file name if the extension differs.
# We keep the path exactly as you wrote (with Hebrew and spaces).
DATA_DIR = Path(r"C:\Users\user\Desktop\BIU דאטה סיינס\ML\פרויקט 2\פרויקט 2 - סופי\Data Cleansing")

# If your file name is "tmdb_tv_after_data_cleansing.csv", set it here.
# If the file doesn’t have the .csv extension in Windows Explorer, add it now.
FILE_BASENAME = "tmdb_tv_after_data_cleansing"
CANDIDATES = [
    DATA_DIR / f"{FILE_BASENAME}.csv",
    DATA_DIR / f"{FILE_BASENAME}.CSV"
]

# ---------- 1.3 Safe loader for CSV (simple, course-aligned) ----------
def load_latest_clean_csv(candidates):
    """
    Try to read the Stage-3 output CSV (comma-separated).
    Keep it simple and aligned with the course: pd.read_csv with utf-8.
    """
    for p in candidates:
        if p.exists():
            try:
                df_ = pd.read_csv(p, encoding="utf-8")
                print(f"[OK] Loaded: {p}")
                return df_
            except UnicodeDecodeError:
                # Fallback if Windows saved with cp1255 or similar
                df_ = pd.read_csv(p, encoding="cp1255")
                print(f"[OK] Loaded with cp1255 encoding: {p}")
                return df_
    raise FileNotFoundError(
        "CSV file not found. Please check the folder and file name (including .csv extension)."
    )

df = load_latest_clean_csv(CANDIDATES)

# ---------- 1.4 Quick integrity checks (as in checklists) ----------
print("Rows, Cols:", df.shape)
print("\nData types & non-null counts:")
print(df.info())

print("\nSample preview:")
display(df.head(3))  # if running in Jupyter; otherwise use print(df.head(3))


[OK] Loaded: C:\Users\user\Desktop\BIU דאטה סיינס\ML\פרויקט 2\פרויקט 2 - סופי\Data Cleansing\tmdb_tv_after_data_cleansing.csv
Rows, Cols: (164705, 65)

Data types & non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164705 entries, 0 to 164704
Data columns (total 65 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id                                164705 non-null  int64  
 1   name                              164705 non-null  object 
 2   number_of_seasons                 164705 non-null  float64
 3   number_of_episodes                164705 non-null  float64
 4   original_language                 164705 non-null  object 
 5   vote_count                        164705 non-null  float64
 6   vote_average                      164705 non-null  float64
 7   overview                          164705 non-null  object 
 8   adult                             164705 non-null  bool   
 9 

,id,name,number_of_seasons,number_of_episodes,original_language,vote_count,vote_average,overview,adult,backdrop_path,...,production_countries_grp,networks_grp,genres_grp,episodes_per_season,popularity_per_vote,first_air_year,series_age,active_days,is_missing_languages,is_missing_production_countries
0,1399,game thrones,8.0,73.0,en,21857.0,8.442,seven noble families fight control mythical la...,False,/2OMB0ynKlyIenMJWI2Dy9IWT4c.jpg,...,Other,Other,Other,9.125000,0.049589,2011.0,14.548939,2954.0,False,False
1,71446,money heist,3.0,41.0,es,17836.0,8.257,carry out biggest heist history mysterious man...,False,/gFZriCkpJYsApPZEF3jhxL4yLzG.jpg,...,Other,Other,Other,13.666667,0.005402,2017.0,8.506502,1676.0,False,False
2,66732,stranger things,4.0,34.0,en,16161.0,8.624,when young boy vanishes small town uncovers my...,False,/2MaumbgBlW1NoPo3ZJO38A6v7OS.jpg,...,United States of America,Netflix,Other,8.500000,0.011491,2016.0,9.303217,2177.0,False,False


In [2]:
#--2--

# =========================================================
#     PROBLEM DEFINITION (Y/X) – Binary Classification
#     Research question: "Will the show have another season?"
#     Y = 1 (continues) / 0 (no additional season)
#     X = eligible, leakage-free features from Stage 3
# =========================================================

# ---------- 2.1 Define Y ----------
# Priority A (per course): use the boolean 'in_production' as the target.
assert "in_production" in df.columns, "Expected 'in_production' column not found."
y = df["in_production"].astype(int)  # True->1, False->0

# Brief documentation string (for your report)
target_doc = (
    "Target (Y) := in_production (True→1, False→0). "
    "Task type := Binary Classification (predict if a show will have another season)."
)
print(target_doc)

# ---------- 2.2 Inspect class balance (very important before splitting) ----------
print("\n[Class balance]")
class_counts = y.value_counts().sort_index()
class_ratio = class_counts / len(y)
print(f"y=0 (no next season): {class_counts.get(0,0)}  | {class_ratio.get(0,0):.2%}")
print(f"y=1 (continues)     : {class_counts.get(1,0)}  | {class_ratio.get(1,0):.2%}")
print(f"Total rows          : {len(y)}")

# A simple imbalance flag (course-level, just to be aware)
minority_rate = class_ratio.min()
if minority_rate < 0.20:
    print("\n[Note] Classes are imbalanced (<20% minority). We will use 'stratify=y' in the split.")

# ---------- 2.3 Define a leakage-free candidate feature list (X) ----------
# Exclude direct identifiers, raw text, URLs/paths, and near-target columns.
exclude_cols = {
    "id", "name", "original_name", "homepage", "backdrop_path", "poster_path",
    "overview",            # raw text — we only use simple lengths later if needed
    "in_production",       # this IS the target
    "status"               # conservative: avoid using as a predictive feature
}

# Prefer Stage-3 prepared features (capped/log/grouped/missingness/derived)
preferred_numeric = [
    "vote_count_capped", "popularity_capped",
    "number_of_episodes_capped", "number_of_seasons_capped",
    "episode_run_time_capped",
    "vote_count_log", "popularity_log", "number_of_episodes_log",
    "episodes_per_season", "popularity_per_vote",
    "first_air_year", "series_age", "active_days"
]
preferred_categoricals = [
    "original_language_grp", "origin_country_grp",
    "production_countries_grp", "networks_grp", "genres_grp"
]
missingness_flags = [
    "vote_average_was_zero", "run_time_was_zero",
    "vote_average_was_missing", "overview_was_missing",
    "backdrop_path_was_missing", "homepage_was_missing",
    "poster_path_was_missing", "tagline_was_missing",
    "genres_was_missing", "created_by_was_missing",
    "languages_was_missing", "networks_was_missing",
    "spoken_languages_was_missing", "production_companies_was_missing",
    "production_countries_was_missing", "episode_run_time_was_missing",
    "is_missing_languages", "is_missing_production_countries"
]

def keep_existing(cols):
    return [c for c in cols if c in df.columns]

base_X_cols = keep_existing(preferred_numeric + preferred_categoricals + missingness_flags)
base_X_cols = [c for c in base_X_cols if c not in exclude_cols]

print("\n[Base X candidate columns]")
print(f"Count: {len(base_X_cols)}")
print(base_X_cols[:25], "..." if len(base_X_cols) > 25 else "")

# ---------- 2.4 (Optional) Quick sanity: ensure no forbidden columns slipped into X ----------
forbidden = {"in_production", "status"}
assert not any(c in forbidden for c in base_X_cols), "Leakage risk: remove target/near-target columns from X."

# From here, we'll move to Stage —3— Train/Test split with stratify=y.


Target (Y) := in_production (True→1, False→0). Task type := Binary Classification (predict if a show will have another season).

[Class balance]
y=0 (no next season): 96892  | 58.83%
y=1 (continues)     : 67813  | 41.17%
Total rows          : 164705

[Base X candidate columns]
Count: 36
['vote_count_capped', 'popularity_capped', 'number_of_episodes_capped', 'number_of_seasons_capped', 'episode_run_time_capped', 'vote_count_log', 'popularity_log', 'number_of_episodes_log', 'episodes_per_season', 'popularity_per_vote', 'first_air_year', 'series_age', 'active_days', 'original_language_grp', 'origin_country_grp', 'production_countries_grp', 'networks_grp', 'genres_grp', 'vote_average_was_zero', 'run_time_was_zero', 'vote_average_was_missing', 'overview_was_missing', 'backdrop_path_was_missing', 'homepage_was_missing', 'poster_path_was_missing'] ...


In [3]:
#--3--

# =========================================================
# CANDIDATE FEATURE SET (From Stage 3) — CLEAN, VISUAL, EXPLAINED
#     GOAL: Curate a transparent "base_feature_candidates" list (not selection yet!)
#     THIS BLOCK DOES NOT CREATE NEW FEATURES — it only documents what's allowed vs. rejected.
# =========================================================

# ---------- 3.0 Overview (what this section does) ----------
# We split columns into clear buckets and explain WHY:
#   - Allowed buckets:
#       * numeric (prefer *_capped, *_log, ratios)
#       * categorical_grouped (*_grp)
#       * date_derived (first_air_year, series_age, active_days)
#       * missingness_flag (e.g., *_was_missing, is_missing_*)
#   - Rejected buckets:
#       * target / near-target (in_production, status)
#       * identifiers / names (id, name, original_name)
#       * raw text & paths/URLs (overview, homepage, poster_path, backdrop_path)
#       * raw multi-value lists (use *_grp instead)

# ---------- 3.1 Define inclusion/exclusion rules (course-aligned) ----------
forbidden_cols = {
    "in_production", "status",               # target / near-target
    "id", "name", "original_name",           # identifiers
    "overview", "homepage", "backdrop_path", "poster_path"  # raw text/paths
}
raw_multi_value = {
    "genres", "created_by", "languages", "networks",
    "origin_country", "spoken_languages",
    "production_companies", "production_countries"
}
preferred_numeric_exact = {"episodes_per_season", "popularity_per_vote",
                           "first_air_year", "series_age", "active_days"}
preferred_numeric_suffixes = ("_capped", "_log")
missingness_prefixes = ("is_missing_",)
missingness_suffixes = ("_was_missing", "_was_zero")
grouped_categorical_suffix = "_grp"

# ---------- 3.2 Classify each column into a bucket & decision ----------
decision_rows = []  # (column, dtype, bucket, include, reason)

def classify_column(col: str):
    """Return (bucket, include, reason) for a given column name (course-aligned logic)."""

    # 1) Hard exclusions
    if col in forbidden_cols:
        return ("excluded", False, "forbidden (target/id/text/url)")
    if col in raw_multi_value:
        return ("excluded", False, "raw multi-value; use grouped version (*_grp)")

    # 2) Missingness flags (transparency)
    if col.startswith(missingness_prefixes) or col.endswith(missingness_suffixes):
        return ("missingness_flag", True, "allowed transparency flag")

    # 3) Grouped categoricals
    if col.endswith(grouped_categorical_suffix):
        return ("categorical_grouped", True, "allowed grouped categorical (*_grp)")

    # 4) Date-derived (from Stage 3)
    if col in {"first_air_year", "series_age", "active_days"}:
        return ("date_derived", True, "allowed date-derived (Stage 3)")

    # 5) Preferred numeric patterns
    if col in preferred_numeric_exact:
        return ("numeric", True, "allowed preferred numeric (exact)")
    if col.endswith(preferred_numeric_suffixes):
        return ("numeric", True, "allowed preferred numeric (suffix)")

    # 6) Plain numeric fallback — allow if truly numeric (int/float/bool)
    if df[col].dtype.kind in {"i", "u", "f", "b"}:
        return ("numeric", True, "allowed numeric (clean fallback)")

    # 7) Default exclusion
    return ("excluded", False, "not in allowed groups (raw text/id/url/multi-value)")

for c in df.columns:
    bucket, include, reason = classify_column(c)
    decision_rows.append((c, str(df[c].dtype), bucket, include, reason))

decision_df = pd.DataFrame(decision_rows, columns=["column", "dtype", "bucket", "include", "reason"])

# ---------- 3.3 Category rules — visual summary (what's allowed vs. rejected) ----------
rules_rows = [
    ("numeric (prepared)",  "Included", "Stable, model-ready numeric variants from Stage 3", 
     "popularity_capped, vote_count_log, episodes_per_season, popularity_per_vote"),
    ("date_derived",        "Included", "Chronological info without peeking into future", 
     "first_air_year, series_age, active_days"),
    ("categorical_grouped", "Included", "High-cardinality grouped to avoid overfitting", 
     "original_language_grp, origin_country_grp, networks_grp, genres_grp"),
    ("missingness_flag",    "Included", "Transparency flags (no imputation leakage)", 
     "overview_was_missing, poster_path_was_missing, is_missing_languages"),
    ("target / near-target","Excluded", "Would leak the answer / direct proxy", 
     "in_production, status"),
    ("identifiers / names", "Excluded", "Non-predictive IDs/strings", 
     "id, name, original_name"),
    ("raw text & paths",    "Excluded", "Not modeled here; URLs/paths add no signal", 
     "overview, homepage, poster_path"),
    ("raw multi-value",     "Excluded", "Use grouped versions (*_grp) instead", 
     "genres, created_by, languages, networks, ..."),
]
category_rules_df = pd.DataFrame(rules_rows, columns=["category", "decision", "why", "examples"])

def color_rule(row):
    return ["background-color: #d4f8d4" if row["decision"] == "Included" else "background-color: #f8d4d4"]*4

display(category_rules_df.style.apply(color_rule, axis=1))

# ---------- 3.4 Full decision table — sorted, color-coded ----------
# Sort: Included first, then by bucket, then by column
decision_df["include_order"] = decision_df["include"].map({True: 0, False: 1})
bucket_order = ["numeric", "categorical_grouped", "date_derived", "missingness_flag", "excluded"]
decision_df["bucket_order"] = decision_df["bucket"].apply(lambda x: bucket_order.index(x) if x in bucket_order else len(bucket_order))
decision_df_sorted = (decision_df
                      .sort_values(["include_order", "bucket_order", "column"])
                      .drop(columns=["include_order", "bucket_order"]))

def color_decision_cell(val):
    return "background-color: #d4f8d4" if val is True else "background-color: #f8d4d4"

display(
    decision_df_sorted[["column","dtype","bucket","include","reason"]]
    .style.applymap(color_decision_cell, subset=["include"])
)

# ---------- 3.5 Included vs. Excluded — separated, with reasons ----------
included_df = decision_df_sorted.query("include == True")[["column","bucket","reason"]]
excluded_df = decision_df_sorted.query("include == False")[["column","bucket","reason"]]

def color_included(_):
    return ["background-color: #e9fbe9"]*3
def color_excluded(_):
    return ["background-color: #fde9e9"]*3

print("Included features (with reasons):", len(included_df))
display(included_df.style.apply(color_included, axis=1))

print("Excluded features (with reasons):", len(excluded_df))
display(excluded_df.style.apply(color_excluded, axis=1))

# ---------- 4.6 Freeze the candidate list for Stage 5 ----------
base_feature_candidates = included_df["column"].tolist()
print("\n[FROZEN] base_feature_candidates:", len(base_feature_candidates))
print(base_feature_candidates[:30], "..." if len(base_feature_candidates) > 30 else "")

# Safety: ensure no forbidden/raw slipped in
assert not any(c in forbidden_cols for c in base_feature_candidates), "Leakage risk: forbidden column found."
assert all(not (c in raw_multi_value) for c in base_feature_candidates), "Found raw multi-value; use *_grp instead."

# OPTIONAL: export decision table for documentation (CSV/HTML)
# decision_df_sorted.to_csv("stage4_candidate_features_decisions.csv", index=False)
# (decision_df_sorted.assign(Decision=decision_df_sorted["include"].map({True:"✅ Included", False:"❌ Excluded"}))
#                     [["column","dtype","bucket","Decision","reason"]]
#                     .to_html("stage4_candidate_features_decisions.html", index=False))


,category,decision,why,examples
0,numeric (prepared),Included,"Stable, model-ready numeric variants from Stage 3","popularity_capped, vote_count_log, episodes_per_season, popularity_per_vote"
1,date_derived,Included,Chronological info without peeking into future,"first_air_year, series_age, active_days"
2,categorical_grouped,Included,High-cardinality grouped to avoid overfitting,"original_language_grp, origin_country_grp, networks_grp, genres_grp"
3,missingness_flag,Included,Transparency flags (no imputation leakage),"overview_was_missing, poster_path_was_missing, is_missing_languages"
4,target / near-target,Excluded,Would leak the answer / direct proxy,"in_production, status"
5,identifiers / names,Excluded,Non-predictive IDs/strings,"id, name, original_name"
6,raw text & paths,Excluded,Not modeled here; URLs/paths add no signal,"overview, homepage, poster_path"
7,raw multi-value,Excluded,Use grouped versions (*_grp) instead,"genres, created_by, languages, networks, ..."


,column,dtype,bucket,include,reason
8,adult,bool,numeric,True,allowed numeric (clean fallback)
28,episode_run_time,float64,numeric,True,allowed numeric (clean fallback)
49,episode_run_time_capped,float64,numeric,True,allowed preferred numeric (suffix)
58,episodes_per_season,float64,numeric,True,allowed preferred numeric (exact)
3,number_of_episodes,float64,numeric,True,allowed numeric (clean fallback)
47,number_of_episodes_capped,float64,numeric,True,allowed preferred numeric (suffix)
52,number_of_episodes_log,float64,numeric,True,allowed preferred numeric (suffix)
2,number_of_seasons,float64,numeric,True,allowed numeric (clean fallback)
48,number_of_seasons_capped,float64,numeric,True,allowed preferred numeric (suffix)
15,popularity,float64,numeric,True,allowed numeric (clean fallback)


Included features (with reasons): 43


,column,bucket,reason
8,adult,numeric,allowed numeric (clean fallback)
28,episode_run_time,numeric,allowed numeric (clean fallback)
49,episode_run_time_capped,numeric,allowed preferred numeric (suffix)
58,episodes_per_season,numeric,allowed preferred numeric (exact)
3,number_of_episodes,numeric,allowed numeric (clean fallback)
47,number_of_episodes_capped,numeric,allowed preferred numeric (suffix)
52,number_of_episodes_log,numeric,allowed preferred numeric (suffix)
2,number_of_seasons,numeric,allowed numeric (clean fallback)
48,number_of_seasons_capped,numeric,allowed preferred numeric (suffix)
15,popularity,numeric,allowed numeric (clean fallback)


Excluded features (with reasons): 22


,column,bucket,reason
9,backdrop_path,excluded,forbidden (target/id/text/url)
21,created_by,excluded,raw multi-value; use grouped version (*_grp)
10,first_air_date,excluded,not in allowed groups (raw text/id/url/multi-value)
20,genres,excluded,raw multi-value; use grouped version (*_grp)
12,homepage,excluded,forbidden (target/id/text/url)
0,id,excluded,forbidden (target/id/text/url)
13,in_production,excluded,forbidden (target/id/text/url)
22,languages,excluded,raw multi-value; use grouped version (*_grp)
11,last_air_date,excluded,not in allowed groups (raw text/id/url/multi-value)
1,name,excluded,forbidden (target/id/text/url)



[FROZEN] base_feature_candidates: 43
['adult', 'episode_run_time', 'episode_run_time_capped', 'episodes_per_season', 'number_of_episodes', 'number_of_episodes_capped', 'number_of_episodes_log', 'number_of_seasons', 'number_of_seasons_capped', 'popularity', 'popularity_capped', 'popularity_log', 'popularity_per_vote', 'vote_average', 'vote_count', 'vote_count_capped', 'vote_count_log', 'genres_grp', 'networks_grp', 'origin_country_grp', 'original_language_grp', 'production_countries_grp', 'active_days', 'first_air_year', 'series_age', 'backdrop_path_was_missing', 'created_by_was_missing', 'episode_run_time_was_missing', 'genres_was_missing', 'homepage_was_missing'] ...


In [4]:
#--4--

# =========================================================
# 5) FEATURE ENGINEERING 
#     We ADD a few transparent features; we DO NOT touch the target.
# =========================================================

# ---------- 5.A TEXT-DERIVED (length-only, as taught) ----------
if "overview" in df.columns:
    df["overview"] = df["overview"].astype("string")
    df["overview_len"] = df["overview"].fillna("").str.len()
    df["overview_words"] = df["overview"].fillna("").str.split().str.len()

if "tagline" in df.columns:
    df["tagline"] = df["tagline"].astype("string")
    df["tagline_len"] = df["tagline"].fillna("").str.len()

print("[5.A] Text-length features created (head):")
cols_show = [c for c in ["overview_len","overview_words","tagline_len"] if c in df.columns]
display(df[cols_show].head(3))

# ---------- 5.B DATE-BASED (safe, no future leakage) ----------
fad = pd.to_datetime(df["first_air_date"], errors="coerce") if "first_air_date" in df.columns else pd.NaT
lad = pd.to_datetime(df["last_air_date"],  errors="coerce") if "last_air_date"  in df.columns else pd.NaT

# Create *_fe columns to keep provenance clear
df["first_air_year_fe"] = fad.dt.year
today = pd.Timestamp.today().normalize()
df["series_age_fe"] = (today - fad).dt.days / 365.25
df["active_days_fe"] = (lad - fad).dt.days

# Date missingness flags (transparency)
df["first_air_date_was_missing"] = fad.isna().astype(int)
df["last_air_date_was_missing"]  = lad.isna().astype(int)

print("[5.B] Date-derived FE (null counts):")
display(df[["first_air_year_fe","series_age_fe","active_days_fe"]].isna().sum())
print("[5.B] Date-derived FE (head):")
display(df[["first_air_year_fe","series_age_fe","active_days_fe"]].head(3))


[5.A] Text-length features created (head):


,overview_len,overview_words,tagline_len
0,265,41,13
1,419,61,15
2,135,18,26


[5.B] Date-derived FE (null counts):


first_air_year_fe    30901
series_age_fe        30901
active_days_fe       31758
dtype: int64

[5.B] Date-derived FE (head):


,first_air_year_fe,series_age_fe,active_days_fe
0,2011.0,14.562628,2954.0
1,2017.0,8.520192,1676.0
2,2016.0,9.316906,2177.0


In [5]:
#--5--

# =========================================================
# 5) FEATURE ENGINEERING (Simple, Course-Aligned, Clean Version)
#     Purpose: Add clear, reproducible, leakage-safe features.
#     We never touch Y ('in_production') and never overwrite Stage-3 columns.
#     Every new feature is documented (why + nulls) and checked for duplication.
# =========================================================

from pandas import Timestamp, to_datetime

# ---------------------------------------------------------
# 5.0 Setup
# ---------------------------------------------------------
fe_log = []  # (feature, category, created, reason, nulls, note)
def _log(name, category, created, reason, note=""):
    fe_log.append((name, category, created, reason, int(df[name].isna().sum()), note))

# Helper: safely create new column if not exists
def _create_feature(name, series, category, reason, note=""):
    if name not in df.columns:
        df[name] = series
        _log(name, category, True, reason, note)
    else:
        _log(name, category, False, "already exists (kept from Stage 3)", note)

# =========================================================
# 5.A) TEXT-DERIVED (length-only, no NLP)
# =========================================================
print("=== 5.A) TEXT-DERIVED FEATURES ===")

if "overview" in df.columns:
    df["overview"] = df["overview"].astype("string")
    _create_feature("overview_len",
                    df["overview"].fillna("").str.len(),
                    "text_length",
                    "number of characters in overview",
                    "numeric text-length feature")
    _create_feature("overview_words",
                    df["overview"].fillna("").str.split().str.len(),
                    "text_length",
                    "word count of overview",
                    "numeric text-length feature")

if "tagline" in df.columns:
    df["tagline"] = df["tagline"].astype("string")
    _create_feature("tagline_len",
                    df["tagline"].fillna("").str.len(),
                    "text_length",
                    "number of characters in tagline",
                    "short-text length feature")

cols_text = [c for c in ["overview_len", "overview_words", "tagline_len"] if c in df.columns]
if cols_text:
    display(df[cols_text].head(3).style.set_caption("Preview — text-derived features"))

# =========================================================
# 5.B) DATE-DERIVED (safe, no future leakage)
# =========================================================
print("\n=== 5.B) DATE-DERIVED FEATURES ===")

fad = to_datetime(df["first_air_date"], errors="coerce") if "first_air_date" in df.columns else None
lad = to_datetime(df["last_air_date"],  errors="coerce")  if "last_air_date"  in df.columns else None

if fad is not None:
    today = Timestamp.today().normalize()

    _create_feature("first_air_year",
                    fad.dt.year,
                    "date_derived",
                    "extract year from first_air_date",
                    "numeric year for modeling")

    _create_feature("series_age",
                    (today - fad).dt.days / 365.25,
                    "date_derived",
                    "years since first_air_date",
                    "series lifetime (no future info)")

    if lad is not None:
        _create_feature("active_days",
                        (lad - fad).dt.days,
                        "date_derived",
                        "days between first_air and last_air",
                        "activity duration")
    else:
        print("No 'last_air_date' column found; skipping active_days.")

    # Missingness flags for dates
    _create_feature("first_air_date_was_missing",
                    fad.isna().astype(int),
                    "missingness_flag",
                    "1 if first_air_date missing else 0")
    if lad is not None:
        _create_feature("last_air_date_was_missing",
                        lad.isna().astype(int),
                        "missingness_flag",
                        "1 if last_air_date missing else 0")
else:
    print("No 'first_air_date' column found; skipping date-derived features.")

cols_date = [c for c in ["first_air_year","series_age","active_days"] if c in df.columns]
if cols_date:
    display(df[cols_date].head(3).style.set_caption("Preview — date-derived features"))

# =========================================================
# 5.C) NUMERIC RATIOS (create only if absent)
# =========================================================
print("\n=== 5.C) NUMERIC RATIO FEATURES ===")

if "episodes_per_season" not in df.columns and {"number_of_episodes","number_of_seasons"} <= set(df.columns):
    nos = df["number_of_seasons"].replace({0: pd.NA})
    _create_feature("episodes_per_season",
                    df["number_of_episodes"] / nos,
                    "numeric_ratio",
                    "episodes / seasons",
                    "pace of production")

if "popularity_per_vote" not in df.columns and {"popularity","vote_count"} <= set(df.columns):
    vc = df["vote_count"].replace({0: pd.NA})
    _create_feature("popularity_per_vote",
                    df["popularity"] / vc,
                    "numeric_ratio",
                    "popularity / vote_count",
                    "relative popularity metric")

cols_ratio = [c for c in ["episodes_per_season","popularity_per_vote"] if c in df.columns]
if cols_ratio:
    display(df[cols_ratio].describe().T.style.set_caption("Numeric ratio features — summary"))

# =========================================================
# 5.D) SAFETY CHECKS & LOG SUMMARY
# =========================================================
print("\n=== 5.D) SAFETY CHECKS ===")
assert "in_production" in df.columns, "Target missing — something went wrong!"
print("✓ Target column untouched.")

# Build a log dataframe for overview
fe_summary = pd.DataFrame(fe_log, columns=["feature","category","created","reason","nulls","note"])
if not fe_summary.empty:
    def color_created(v):
        return "background-color:#d4f8d4" if v else "background-color:#eeeeee"
    display(fe_summary.style.applymap(color_created, subset=["created"])
            .set_caption("Feature Engineering — Created vs Existing (with reasons)"))

# =========================================================
# 5.E) UPDATE BASE FEATURE CANDIDATES (no duplicates)
# =========================================================
print("\n=== 5.E) UPDATE BASE FEATURE CANDIDATES ===")

# Start from Stage-4 approved columns
base_stage4 = decision_df.query("include == True and bucket != 'excluded'")["column"].tolist()

# FE additions allowed in course
fe_allowed = [
    "overview_len", "overview_words", "tagline_len",
    "first_air_date_was_missing", "last_air_date_was_missing",
    "episodes_per_season", "popularity_per_vote"
]
fe_allowed = [c for c in fe_allowed if c in df.columns]

# Merge, drop any duplicates safely
base_feature_candidates = sorted(dict.fromkeys(base_stage4 + fe_allowed))

print("Final feature candidate count:", len(base_feature_candidates))
display(pd.DataFrame({"feature_candidates": base_feature_candidates}).head(20)
        .style.set_caption("First 20 feature candidates (final Stage-5 list)"))

# End of Stage-5 — next per checklist:
#   - Train/Test split (Section 3)
#   - Encoding & Scaling (Section 6)
#   - Feature Selection (Section 7)


=== 5.A) TEXT-DERIVED FEATURES ===


,overview_len,overview_words,tagline_len
0,265,41,13
1,419,61,15
2,135,18,26



=== 5.B) DATE-DERIVED FEATURES ===


,first_air_year,series_age,active_days
0,2011.000000,14.548939,2954.000000
1,2017.000000,8.506502,1676.000000
2,2016.000000,9.303217,2177.000000



=== 5.C) NUMERIC RATIO FEATURES ===


,count,mean,std,min,25%,50%,75%,max
episodes_per_season,164705.000000,12.554012,32.135506,0.000000,1.000000,6.000000,12.666667,3919.000000
popularity_per_vote,164705.000000,1.863190,15.977250,0.000000,0.600000,0.600000,1.176000,2148.789000



=== 5.D) SAFETY CHECKS ===
✓ Target column untouched.


,feature,category,created,reason,nulls,note
0,overview_len,text_length,False,already exists (kept from Stage 3),0,numeric text-length feature
1,overview_words,text_length,False,already exists (kept from Stage 3),0,numeric text-length feature
2,tagline_len,text_length,False,already exists (kept from Stage 3),0,short-text length feature
3,first_air_year,date_derived,False,already exists (kept from Stage 3),30901,numeric year for modeling
4,series_age,date_derived,False,already exists (kept from Stage 3),30908,series lifetime (no future info)
5,active_days,date_derived,False,already exists (kept from Stage 3),31823,activity duration
6,first_air_date_was_missing,missingness_flag,False,already exists (kept from Stage 3),0,
7,last_air_date_was_missing,missingness_flag,False,already exists (kept from Stage 3),0,



=== 5.E) UPDATE BASE FEATURE CANDIDATES ===
Final feature candidate count: 48


,feature_candidates
0,active_days
1,adult
2,backdrop_path_was_missing
3,created_by_was_missing
4,episode_run_time
5,episode_run_time_capped
6,episode_run_time_was_missing
7,episodes_per_season
8,first_air_date_was_missing
9,first_air_year


In [6]:
#--6--

# =========================================================
# 6) TRAIN/TEST SPLIT (after FE, before Encoding/Selection) — Avoid Leakage
#     - Build X from final Stage-5 candidates (leakage-safe)
#     - y = in_production (True→1, False→0)
#     - Hold-out split 80/20 with stratify=y
#     - Keep TEST untouched for final evaluation
# NOTE: Assumes from sklearn.model_selection import train_test_split was imported at the top.
# =========================================================

# ---------- 6.1 Safety: validate candidate list & remove forbidden just in case ----------
forbidden_cols_set = {
    "in_production", "status",       # target / near-target
    "id", "name", "original_name",   # identifiers
    "overview", "homepage", "backdrop_path", "poster_path"  # raw text/paths
}
assert 'base_feature_candidates' in globals() and len(base_feature_candidates) > 0, \
    "Stage-5 candidates not found. Please run Stage-5 first."

leak_intersection = sorted(set(base_feature_candidates) & forbidden_cols_set)
assert len(leak_intersection) == 0, f"Forbidden columns in candidates: {leak_intersection}"

# ---------- 6.2 Build X, y ----------
X = df[base_feature_candidates].copy()
y = df['in_production'].astype(int)  # True→1, False→0

# Convert boolean features (if any) to numeric 0/1 to keep model-ready dtypes
bool_cols = [c for c in X.columns if str(X[c].dtype) == 'bool']
if bool_cols:
    X[bool_cols] = X[bool_cols].astype('int8')

# ---------- 6.3 Split 80/20 with stratify ----------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

# (Optional for reporting only) keep ids to join predictions later — NOT a feature
ids_train = df.loc[X_train.index, 'id'] if 'id' in df.columns else None
ids_test  = df.loc[X_test.index,  'id'] if 'id' in df.columns else None

# ---------- 6.4 Integrity checks & visual summaries ----------
print("=== 6) TRAIN/TEST SPLIT — integrity checks ===")
print(f"Train shape: {X_train.shape}   |   Test shape: {X_test.shape}")

# Class balance check (stratify)
train_pos, test_pos = y_train.mean(), y_test.mean()
print(f"Train class ratio (1's): {train_pos:.2%}")
print(f"Test  class ratio (1's): {test_pos:.2%}")
print("Index overlap (should be 0):", len(set(X_train.index) & set(X_test.index)))

# Basic dtype audit — ensures model-ready numerics
dtype_summary = X_train.dtypes.astype(str).value_counts().to_frame('count')
display(dtype_summary.style.set_caption("X_train dtypes — count by dtype"))

# Missingness overview on TRAIN
nulls_train = X_train.isna().sum().sort_values(ascending=False)
display(nulls_train[nulls_train > 0].to_frame("nulls").head(20)
        .style.set_caption("Top TRAIN columns with missing values (if any)"))

# Peek a few rows/cols for sanity
cols_peek = list(X_train.columns[: min(10, X_train.shape[1])])
display(X_train[cols_peek].head(3).style.set_caption("X_train preview (first 10 features)"))

print("✓ Split complete. Proceed to Encoding (One-Hot) on TRAIN only (apply same encoder to TEST).")


#--6.5--

# =========================================================
# 6.5) ENCODING (fit on TRAIN only, then apply to TEST)
#     - Identify grouped categoricals (*_grp) from base_feature_candidates
#     - Impute numeric (median) & categorical (most_frequent) on TRAIN only
#     - OneHotEncoder(handle_unknown='ignore'), fit on TRAIN only
#     - Build X_train_prepared, X_test_prepared, final_feature_names
#     - Save small artifacts dict for later pickle
# NOTE: requires at top: from sklearn.preprocessing import OneHotEncoder
#       (SimpleImputer כבר יובא למעלה אצלך)
# =========================================================

# 1) Identify columns
cat_cols = [c for c in base_feature_candidates if c.endswith("_grp")]
num_cols = [c for c in base_feature_candidates if c not in cat_cols]

print("Categorical to encode:", cat_cols)
print("Numeric kept (after impute):", len(num_cols), "columns")

# 2) Imputers (fit on TRAIN only)
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

# חשוב: המרות לטיפוס מחרוזת עבור OHE (מונע תקלות)
if cat_cols:
    X_train.loc[:, cat_cols] = X_train[cat_cols].astype("string")
    X_test.loc[:,  cat_cols] = X_test[cat_cols].astype("string")

X_train_num_imp = num_imputer.fit_transform(X_train[num_cols]) if num_cols else np.empty((len(X_train), 0))
X_test_num_imp  = num_imputer.transform(X_test[num_cols])      if num_cols else np.empty((len(X_test), 0))

X_train_cat_imp = cat_imputer.fit_transform(X_train[cat_cols]) if cat_cols else np.empty((len(X_train), 0))
X_test_cat_imp  = cat_imputer.transform(X_test[cat_cols])      if cat_cols else np.empty((len(X_test), 0))

# 3) One-Hot Encoder (fit on TRAIN only)
#    הערה: בגרסאות חדשות של sklearn הפרמטר הוא sparse_output, בישנות זה sparse.
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

X_train_cat_enc = ohe.fit_transform(X_train_cat_imp) if cat_cols else np.empty((len(X_train), 0))
X_test_cat_enc  = ohe.transform(X_test_cat_imp)      if cat_cols else np.empty((len(X_test), 0))

ohe_feature_names = list(ohe.get_feature_names_out(cat_cols)) if cat_cols else []

# 4) Reassemble final matrices (+ names)
X_train_prepared = np.hstack([X_train_num_imp, X_train_cat_enc]).astype(float)
X_test_prepared  = np.hstack([X_test_num_imp,  X_test_cat_enc]).astype(float)

final_feature_names = list(num_cols) + ohe_feature_names

# 5) Sanity prints
print("=== 6.5) ENCODING — summary ===")
print("Numeric block shape (train/test):", X_train_num_imp.shape, "/", X_test_num_imp.shape)
print("Categorical block shape after OHE (train/test):", X_train_cat_enc.shape, "/", X_test_cat_enc.shape)
print("Final prepared shapes (train/test):", X_train_prepared.shape, "/", X_test_prepared.shape)
print("Final feature count:", len(final_feature_names))
print("NaNs in X_train_prepared:", np.isnan(X_train_prepared).sum())
print("NaNs in X_test_prepared :", np.isnan(X_test_prepared).sum())

# 6) Artifacts for later pickle
encoders_artifacts = {
    "num_imputer": num_imputer,
    "cat_imputer": cat_imputer,
    "ohe": ohe,
    "final_feature_names": final_feature_names,
    "ohe_feature_names": ohe_feature_names,
    "cat_cols": cat_cols,
    "num_cols": num_cols,
}


=== 6) TRAIN/TEST SPLIT — integrity checks ===
Train shape: (131764, 48)   |   Test shape: (32941, 48)
Train class ratio (1's): 41.17%
Test  class ratio (1's): 41.17%
Index overlap (should be 0): 0


,count
float64,21
int64,17
object,5
int8,3
Int64,2


,nulls
active_days,25462
series_age,24749
first_air_year,24742


,active_days,adult,backdrop_path_was_missing,created_by_was_missing,episode_run_time,episode_run_time_capped,episode_run_time_was_missing,episodes_per_season,first_air_date_was_missing,first_air_year
97662,nan,0,1,1,60.000000,60.000000,0,0.000000,1,nan
63217,28.000000,0,0,1,42.000000,42.000000,1,5.000000,0,2014.000000
90912,28.000000,0,0,0,60.000000,60.000000,0,5.000000,0,2011.000000


✓ Split complete. Proceed to Encoding (One-Hot) on TRAIN only (apply same encoder to TEST).
Categorical to encode: ['genres_grp', 'networks_grp', 'origin_country_grp', 'original_language_grp', 'production_countries_grp']
Numeric kept (after impute): 43 columns
=== 6.5) ENCODING — summary ===
Numeric block shape (train/test): (131764, 43) / (32941, 43)
Categorical block shape after OHE (train/test): (131764, 65) / (32941, 65)
Final prepared shapes (train/test): (131764, 108) / (32941, 108)
Final feature count: 108
NaNs in X_train_prepared: 0
NaNs in X_test_prepared : 0


In [7]:
#--7--

# =========================================================
# 7) FEATURE SELECTION (Classification) — Full, single block
#     Inputs (must already exist from previous steps):
#       - X_train_prepared, X_test_prepared   # numeric+OHE matrices from Stage 6 (Encoding)
#       - final_feature_names                 # names aligned to X_*_prepared columns
#       - y_train                             
#     Optional (if computed earlier):
#       - comp["in_all_3"]  # 'core' features intersection from the 3-rule comparison
#
#     Outputs:
#       - scores (DataFrame: feature, f_score, p_value) — ranked by f_score desc
#       - selected_feature_names  (FINAL Top-30 after de-correlation)
#       - X_train_selected, X_test_selected   (aligned to selected_feature_names)
#       - CSV: Stage4_selected_features_FINAL30_decorrelated.csv
#
#     Steps:
#       7.1 Remove zero-variance features (VarianceThreshold)
#       7.2 Univariate ANOVA ranking (SelectKBest with k='all')
#       7.3 Choose K=30 via a transparent rule (Core + Best-by-F; fall back to Top-30)
#       7.4 Correlation pruning (|r|≥0.95) + refill to K=30 by F-score
#       7.5 Sanity checks (shapes, NaNs, forbidden columns)
#       7.6 Export final names to CSV
# =========================================================


# ---------- 7.1 VarianceThreshold (fit on TRAIN only) ----------
print("=== 7.1) VarianceThreshold ===")
vt = VarianceThreshold(threshold=0.0)
X_train_vt = vt.fit_transform(X_train_prepared)
X_test_vt  = vt.transform(X_test_prepared)
mask_vt = vt.get_support()
features_vt = np.array(final_feature_names)[mask_vt]
print(f"Kept after VT: {X_train_vt.shape[1]} / {len(final_feature_names)} features")

# ---------- 7.2 ANOVA (SelectKBest with k='all') on TRAIN ----------
print("\n=== 7.2) ANOVA F-score ranking (train-only) ===")
skb = SelectKBest(score_func=f_classif, k='all')
_ = skb.fit(X_train_vt, y_train)

scores = pd.DataFrame({
    "feature": features_vt,
    "f_score": skb.scores_,
    "p_value": skb.pvalues_
}).sort_values("f_score", ascending=False).reset_index(drop=True)

# (Optional) quick peek
display(scores.head(20).style.set_caption("Top 20 features by ANOVA F-score (higher = better)"))

# ---------- 7.3 Choose K=30 via a transparent rule ----------
# Rule: if a 'core' list exists (features present in all 3 selection rules), keep all core first,
# then fill by F-score until K=30; otherwise, simply take Top-30 by F-score.
print("\n=== 7.3) Final-K rule (K=30): Core + Best-by-F (fallback: Top-30) ===")
FINAL_K = 30

if 'comp' in globals() and 'in_all_3' in comp.columns:
    core = comp.loc[comp["in_all_3"] == 1, "feature"].tolist()
    print(f"Core features found: {len(core)}")
else:
    core = []
    print("No 'core' (in_all_3) available — using pure Top-30 by F-score.")

pre30 = list(core)
for f in scores["feature"]:
    if len(pre30) == FINAL_K: break
    if f not in pre30:
        pre30.append(f)
pre30 = pre30[:FINAL_K]

# Map to indices and build current Top-30 matrices (pre-correlation)
feat_to_idx = {f: i for i, f in enumerate(features_vt)}
idx_pre30 = [feat_to_idx[f] for f in pre30 if f in feat_to_idx]
X_train_sel = X_train_vt[:, idx_pre30]
X_test_sel  = X_test_vt[:,  idx_pre30]
selected_names_pre_corr = pre30

print("K=30 (pre-correlation) — first 12:")
print(selected_names_pre_corr[:12], "...")

# ---------- 7.4 Correlation pruning (|r|≥0.95) + refill to 30 ----------
print("\n=== 7.4) Correlation pruning (|r|≥0.95) + refill to K=30 ===")
CORR_THRESHOLD = 0.95

Xsel_train_df = pd.DataFrame(X_train_sel, columns=selected_names_pre_corr)

# Consider only "original numeric-like" (exclude one-hot dummies by name)
numeric_like = [f for f in selected_names_pre_corr if not ("_grp_" in f or f.endswith("_grp"))]
corr = Xsel_train_df[numeric_like].corr(method='pearson')

pairs = []
for i, a in enumerate(numeric_like):
    for j, b in enumerate(numeric_like[i+1:], start=i+1):
        r = corr.iloc[i, j]  # correlation(a,b)
        if abs(r) >= CORR_THRESHOLD:
            pairs.append((a, b, float(r)))

if pairs:
    print(f"Found {len(pairs)} highly-correlated numeric pairs (|r|≥{CORR_THRESHOLD}):")
    for a, b, r in sorted(pairs, key=lambda t: -abs(t[2])):
        print(f" - {a} ↔ {b} (r={r:.2f})")
else:
    print("No numeric pairs exceeding the correlation threshold.")

# Tie-break rules (transparent and deterministic)
score_map = dict(zip(scores["feature"], scores["f_score"]))

def choose_to_drop(a, b):
    # Prefer *_was_missing over *_was_zero
    if a.endswith("_was_zero") and b.endswith("_was_missing"): return a
    if b.endswith("_was_zero") and a.endswith("_was_missing"): return b
    # Prefer series_age over first_air_year
    if {"first_air_year","series_age"} == {a,b}: return "first_air_year"
    # Prefer popularity_log over popularity_capped
    if {"popularity_log","popularity_capped"} == {a,b}: return "popularity_capped"
    # Else: lower ANOVA F-score drops
    fa, fb = score_map.get(a, -1), score_map.get(b, -1)
    if fa < fb: return a
    if fb < fa: return b
    # Deterministic fallback on tie
    return sorted([a,b])[1]

to_drop = set()
for a, b, r in pairs:
    to_drop.add(choose_to_drop(a, b))

kept = [f for f in selected_names_pre_corr if f not in to_drop]

# Refill back to K=30 strictly by F-score order (skip dropped and already kept)
for f in scores["feature"]:
    if len(kept) >= FINAL_K: break
    if f not in kept and f not in to_drop:
        kept.append(f)
kept = kept[:FINAL_K]

# Update matrices according to 'kept'
idx_final = [feat_to_idx[f] for f in kept if f in feat_to_idx]
X_train_selected = X_train_vt[:, idx_final]
X_test_selected  = X_test_vt[:,  idx_final]
selected_feature_names = kept

print("\nDropped due to high correlation:", sorted(to_drop))
print("Final K after pruning + refill:", len(selected_feature_names))
print("Final Top-30 (first 15):", selected_feature_names[:15], "...")

# ---------- 7.5 Sanity checks ----------
print("\n=== 7.5) Sanity checks ===")
forbidden = {"in_production", "status", "id", "name", "original_name", "overview",
             "homepage", "backdrop_path", "poster_path"}
print("Forbidden in selected:", sorted(set(selected_feature_names) & forbidden))
print("NaNs in X_train_selected:", np.isnan(X_train_selected).sum())
print("NaNs in X_test_selected :", np.isnan(X_test_selected).sum())
print("Shapes (train/test):", X_train_selected.shape, "/", X_test_selected.shape)

# ---------- 7.6 Export final selected names (CSV) ----------
out_csv = "Stage4_selected_features_FINAL30_decorrelated.csv"
pd.Series(selected_feature_names, name="final30").to_csv(out_csv, index=False)
print(f"Saved: {out_csv}")


=== 7.1) VarianceThreshold ===
Kept after VT: 106 / 108 features

=== 7.2) ANOVA F-score ranking (train-only) ===


,feature,f_score,p_value
0,episode_run_time_was_missing,12511.402806,0.000000
1,run_time_was_zero,12511.402806,0.000000
2,first_air_year,12064.604277,0.000000
3,series_age,11953.114920,0.000000
4,last_air_date_was_missing,5147.133388,0.000000
5,overview_words,5103.695454,0.000000
6,overview_len,5042.510285,0.000000
7,networks_was_missing,4567.368282,0.000000
8,networks_grp_Unknown,4567.368282,0.000000
9,spoken_languages_was_missing,4370.779321,0.000000



=== 7.3) Final-K rule (K=30): Core + Best-by-F (fallback: Top-30) ===
No 'core' (in_all_3) available — using pure Top-30 by F-score.
K=30 (pre-correlation) — first 12:
['episode_run_time_was_missing', 'run_time_was_zero', 'first_air_year', 'series_age', 'last_air_date_was_missing', 'overview_words', 'overview_len', 'networks_was_missing', 'networks_grp_Unknown', 'spoken_languages_was_missing', 'languages_was_missing', 'first_air_date_was_missing'] ...

=== 7.4) Correlation pruning (|r|≥0.95) + refill to K=30 ===
Found 6 highly-correlated numeric pairs (|r|≥0.95):
 - episode_run_time_was_missing ↔ run_time_was_zero (r=1.00)
 - vote_average_was_missing ↔ vote_average_was_zero (r=1.00)
 - first_air_year ↔ series_age (r=-1.00)
 - overview_words ↔ overview_len (r=1.00)
 - spoken_languages_was_missing ↔ languages_was_missing (r=0.99)
 - popularity_log ↔ popularity_capped (r=0.98)

Dropped due to high correlation: ['first_air_year', 'languages_was_missing', 'overview_len', 'popularity_capped

In [8]:
#--8--

# =========================================================
# 8) FEATURE SELECTION (Train-only) — Single, Clean Block
#     Inputs (from previous stages):
#       - X_train_prepared, X_test_prepared, y_train, y_test
#       - final_feature_names (aligned to X_*_prepared)
#     Outputs:
#       - scores (DF: feature, f_score, p_value) ranked desc
#       - selected_feature_names (FINAL Top-30 after de-correlation)
#       - X_train_selected, X_test_selected
#       - CSVs: Stage4_selected_features_FINAL30.csv,
#               Stage4_selected_features_FINAL30_decorrelated.csv,
#               Stage4_selected_features_FINAL30_with_buckets.csv
#
#     Steps (as per checklist):
#       8.1 VarianceThreshold (remove zero-variance)
#       8.2 ANOVA F-score (k='all') — ranking
#       8.3 Build Top-30 with transparent rule:
#           - If "core" exists (intersection of 3 rules) ⇒ core + best-by-F
#           - Else ⇒ pure Top-30 by F-score
#           (Computes the 3-rule comparison inside this block)
#       8.4 Correlation pruning (|r|≥0.95) + refill to K=30
#       8.5 Documentation table (bucket & why) + category breakdown
#       8.6 Sanity checks + export CSVs
# =========================================================


# ---------- 8.1 VarianceThreshold ----------
print("=== 8.1) VarianceThreshold ===")
vt = VarianceThreshold(threshold=0.0)
X_train_vt = vt.fit_transform(X_train_prepared)   # fit on TRAIN
X_test_vt  = vt.transform(X_test_prepared)        # transform TEST with same mask
mask_vt = vt.get_support()
features_vt = np.array(final_feature_names)[mask_vt]
print(f"Kept after VT: {X_train_vt.shape[1]} / {len(final_feature_names)} features")

# ---------- 8.2 ANOVA F-score (k='all') ----------
print("\n=== 8.2) ANOVA F-score ranking (train-only) ===")
skb = SelectKBest(score_func=f_classif, k='all')
_ = skb.fit(X_train_vt, y_train)

scores = (pd.DataFrame({
    "feature": features_vt,
    "f_score": skb.scores_,
    "p_value": skb.pvalues_
})
 .sort_values("f_score", ascending=False)
 .reset_index(drop=True))

# Peek
try:
    display(scores.head(20).style.set_caption("Top 20 features by ANOVA F-score"))
except:
    pass

# ---------- 8.3 Build Top-30 with transparent rule ----------
# Three-rule comparison (A: fixed top-40, B: p<=0.05 (cap 40), C: elbow 85% (cap 40))
print("\n=== 8.3) Final-K rule (K=30): Core + Best-by-F (fallback: Top-30) ===")
MAX_CAP = 40
def pick_A_fixed_topK(df, K=MAX_CAP): return df.head(min(K, len(df)))['feature'].tolist()
def pick_B_pvalue_cap(df, p_thresh=0.05, cap=MAX_CAP):
    sig = df[df['p_value'] <= p_thresh]
    return sig.head(min(cap, len(sig)))['feature'].tolist() if len(sig) > 0 else pick_A_fixed_topK(df, cap)
def pick_C_elbow_coverage(df, coverage=0.85, cap=MAX_CAP):
    s = df['f_score'].clip(lower=0).values
    total = s.sum() if s.sum() > 0 else 1.0
    idx = int(np.searchsorted(np.cumsum(s)/total, coverage))
    K = min(idx+1, cap, len(df))
    return df.head(K)['feature'].tolist()

sel_A = pick_A_fixed_topK(scores, MAX_CAP)
sel_B = pick_B_pvalue_cap(scores, 0.05, MAX_CAP)
sel_C = pick_C_elbow_coverage(scores, 0.85, MAX_CAP)
core = list(set(sel_A) & set(sel_B) & set(sel_C))

print(f"Core features found: {len(core)}")

FINAL_K = 30
pre30 = list(core)
for f in scores["feature"]:
    if len(pre30) == FINAL_K: break
    if f not in pre30:
        pre30.append(f)
pre30 = pre30[:FINAL_K]

# Map to indices and slice (pre-correlation)
feat_to_idx = {f: i for i, f in enumerate(features_vt)}
idx_pre30 = [feat_to_idx[f] for f in pre30 if f in feat_to_idx]
X_train_sel = X_train_vt[:, idx_pre30]
X_test_sel  = X_test_vt[:,  idx_pre30]
selected_names_pre_corr = pre30

print("K=30 (pre-correlation) — first 12:")
print(selected_names_pre_corr[:12], "...")

# ---------- 8.4 Correlation pruning (|r|≥0.95) + refill to 30 ----------
print("\n=== 8.4) Correlation pruning (|r|≥0.95) + refill to K=30 ===")
CORR_THRESHOLD = 0.95
Xsel_train_df = pd.DataFrame(X_train_sel, columns=selected_names_pre_corr)

# numeric-like = exclude one-hot indicators by name pattern
numeric_like = [f for f in selected_names_pre_corr if not ("_grp_" in f or f.endswith("_grp"))]
corr = Xsel_train_df[numeric_like].corr(method='pearson')

pairs = []
for i, a in enumerate(numeric_like):
    for b in numeric_like[i+1:]:
        r = corr.loc[a, b]
        if abs(r) >= CORR_THRESHOLD:
            pairs.append((a, b, float(r)))

if pairs:
    print(f"Found {len(pairs)} highly-correlated numeric pairs (|r|≥{CORR_THRESHOLD}):")
    for a, b, r in sorted(pairs, key=lambda t: -abs(t[2])):
        print(f" - {a} ↔ {b} (r={r:.2f})")
else:
    print("No numeric pairs exceeding the correlation threshold.")

score_map = dict(zip(scores["feature"], scores["f_score"]))

def choose_to_drop(a, b):
    # Transparent tie-break rules:
    if a.endswith("_was_zero") and b.endswith("_was_missing"): return a
    if b.endswith("_was_zero") and a.endswith("_was_missing"): return b
    if {"first_air_year","series_age"} == {a,b}: return "first_air_year"
    if {"popularity_log","popularity_capped"} == {a,b}: return "popularity_capped"
    fa, fb = score_map.get(a, -1), score_map.get(b, -1)
    if fa < fb: return a
    if fb < fa: return b
    return sorted([a,b])[1]  # deterministic fallback

to_drop = set(choose_to_drop(a, b) for a, b, _ in pairs)
kept = [f for f in selected_names_pre_corr if f not in to_drop]

# Refill strictly by F-score (skip dropped & already kept)
for f in scores["feature"]:
    if len(kept) >= FINAL_K: break
    if f not in kept and f not in to_drop:
        kept.append(f)
kept = kept[:FINAL_K]

# Update matrices and names
idx_final = [feat_to_idx[f] for f in kept if f in feat_to_idx]
X_train_selected = X_train_vt[:, idx_final]
X_test_selected  = X_test_vt[:,  idx_final]
selected_feature_names = kept

print("\nDropped due to high correlation:", sorted(to_drop))
print("Final K after pruning + refill:", len(selected_feature_names))
print("Final Top-30 (first 15):", selected_feature_names[:15], "...")

# ---------- 8.5 Documentation table (buckets & why) ----------
def _bucket(f):
    if f.endswith("_grp") or "_grp_" in f: return "categorical (one-hot)"
    if f.endswith("_was_missing") or f.startswith("is_missing_") or f in ["first_air_date_was_missing","last_air_date_was_missing"]:
        return "missingness flag"
    if any(s in f for s in ["_log","_capped","_per_"]): return "numeric (prepared)"
    if f in ["first_air_year","series_age","active_days"]: return "date-derived"
    if f in ["overview_len","overview_words","tagline_len"]: return "text length"
    return "numeric (plain)"

doc_df = pd.DataFrame({
    "feature": selected_feature_names,
    "bucket":  [_bucket(f) for f in selected_feature_names]
})
doc_df["why"] = doc_df["bucket"].map({
    "categorical (one-hot)": "platform/language/genre signal",
    "missingness flag":      "transparency; absence can be predictive",
    "numeric (prepared)":    "stabilized popularity/size ratios (log/cap)",
    "date-derived":          "chronology without future leakage",
    "text length":           "informative text proxy without NLP",
    "numeric (plain)":       "core counts/durations"
})

try:
    display(doc_df.style.set_caption("FINAL Top-30 — feature categories & rationale"))
    display(doc_df["bucket"].value_counts().to_frame("count").style.set_caption("Category breakdown (Top-30)"))
except:
    pass

# ---------- 8.6 Sanity checks + export ----------
print("\n=== 8.6) Sanity checks ===")
forbidden = {"in_production","status","id","name","original_name","overview","homepage","backdrop_path","poster_path"}
print("Forbidden in selected:", sorted(set(selected_feature_names) & forbidden))
print("NaNs in X_train_selected:", np.isnan(X_train_selected).sum())
print("NaNs in X_test_selected :", np.isnan(X_test_selected).sum())
print("Shapes (train/test):", X_train_selected.shape, "/", X_test_selected.shape)

# Exports (documentation)
pd.Series(selected_feature_names, name="final30").to_csv("Stage4_selected_features_FINAL30.csv", index=False)
pd.Series(selected_feature_names, name="final30_decorrelated").to_csv("Stage4_selected_features_FINAL30_decorrelated.csv", index=False)
doc_df.to_csv("Stage4_selected_features_FINAL30_with_buckets.csv", index=False)
print("Saved: Stage4_selected_features_FINAL30.csv")
print("Saved: Stage4_selected_features_FINAL30_decorrelated.csv")
print("Saved: Stage4_selected_features_FINAL30_with_buckets.csv")


=== 8.1) VarianceThreshold ===
Kept after VT: 106 / 108 features

=== 8.2) ANOVA F-score ranking (train-only) ===


,feature,f_score,p_value
0,episode_run_time_was_missing,12511.402806,0.000000
1,run_time_was_zero,12511.402806,0.000000
2,first_air_year,12064.604277,0.000000
3,series_age,11953.114920,0.000000
4,last_air_date_was_missing,5147.133388,0.000000
5,overview_words,5103.695454,0.000000
6,overview_len,5042.510285,0.000000
7,networks_was_missing,4567.368282,0.000000
8,networks_grp_Unknown,4567.368282,0.000000
9,spoken_languages_was_missing,4370.779321,0.000000



=== 8.3) Final-K rule (K=30): Core + Best-by-F (fallback: Top-30) ===
Core features found: 29
K=30 (pre-correlation) — first 12:
['number_of_seasons', 'overview_was_missing', 'overview_len', 'networks_grp_Other', 'popularity_log', 'first_air_year', 'vote_average_was_missing', 'poster_path_was_missing', 'vote_count_log', 'run_time_was_zero', 'genres_was_missing', 'original_language_grp_en'] ...

=== 8.4) Correlation pruning (|r|≥0.95) + refill to K=30 ===
Found 6 highly-correlated numeric pairs (|r|≥0.95):
 - vote_average_was_missing ↔ vote_average_was_zero (r=1.00)
 - run_time_was_zero ↔ episode_run_time_was_missing (r=1.00)
 - first_air_year ↔ series_age (r=-1.00)
 - overview_len ↔ overview_words (r=1.00)
 - spoken_languages_was_missing ↔ languages_was_missing (r=0.99)
 - popularity_log ↔ popularity_capped (r=0.98)

Dropped due to high correlation: ['first_air_year', 'languages_was_missing', 'overview_len', 'popularity_capped', 'run_time_was_zero', 'vote_average_was_zero']
Final K af

,feature,bucket,why
0,number_of_seasons,numeric (plain),core counts/durations
1,overview_was_missing,missingness flag,transparency; absence can be predictive
2,networks_grp_Other,categorical (one-hot),platform/language/genre signal
3,popularity_log,numeric (prepared),stabilized popularity/size ratios (log/cap)
4,vote_average_was_missing,missingness flag,transparency; absence can be predictive
5,poster_path_was_missing,missingness flag,transparency; absence can be predictive
6,vote_count_log,numeric (prepared),stabilized popularity/size ratios (log/cap)
7,genres_was_missing,missingness flag,transparency; absence can be predictive
8,original_language_grp_en,categorical (one-hot),platform/language/genre signal
9,created_by_was_missing,missingness flag,transparency; absence can be predictive


,count
bucket,
categorical (one-hot),13
missingness flag,11
numeric (prepared),3
numeric (plain),1
date-derived,1
text length,1



=== 8.6) Sanity checks ===
Forbidden in selected: []
NaNs in X_train_selected: 0
NaNs in X_test_selected : 0
Shapes (train/test): (131764, 30) / (32941, 30)
Saved: Stage4_selected_features_FINAL30.csv
Saved: Stage4_selected_features_FINAL30_decorrelated.csv
Saved: Stage4_selected_features_FINAL30_with_buckets.csv


In [9]:
#--9--

# =========================================================
# 9) QUALITY & LEAKAGE CHECKS
# =========================================================

forbidden = {"in_production", "status", "id", "name", "original_name", "overview",
             "homepage", "backdrop_path", "poster_path"}

print("Forbidden in selected features:", sorted(set(selected_feature_names) & forbidden))

print("NaNs in X_train_selected:", np.isnan(X_train_selected).sum())
print("NaNs in X_test_selected :", np.isnan(X_test_selected).sum())
print("Target present & untouched:", "in_production" in df.columns, "| dtype:", df["in_production"].dtype)

print("Train/Test shapes:", X_train_selected.shape, "/", X_test_selected.shape)
print("Consistency check:", X_train_selected.shape[1] == X_test_selected.shape[1])


Forbidden in selected features: []
NaNs in X_train_selected: 0
NaNs in X_test_selected : 0
Target present & untouched: True | dtype: bool
Train/Test shapes: (131764, 30) / (32941, 30)
Consistency check: True


In [11]:
#--10--

# =========================================================
# 10) SAVE ARTIFACTS (Pickle) — Stage 4 (Feature Eng. & Selection)
#     Saved objects:
#       - num_imputer, cat_imputer, ohe       (fitted on TRAIN)
#       - vt (VarianceThreshold)              (fitted on TRAIN)
#       - final_feature_names                 (after encoding; pre-VT)
#       - selected_feature_names (FINAL30)    (for modeling)
#       - selection_policy metadata           (for documentation)
# =========================================================

artifacts = {
    "num_imputer": num_imputer,
    "cat_imputer": cat_imputer,
    "ohe": ohe,
    "vt": vt,
    "final_feature_names": final_feature_names,      # all features after encoding (numeric + one-hot)
    "selected_feature_names": selected_feature_names,  # FINAL Top-30
    "selection_policy": {
        "vt_threshold": 0.0,
        "anova_rule": "rank by F-score on TRAIN only",
        "final_rule": "core (in all 3) + best remainder by F-score up to 30"
    },
    "shapes": {
        "X_train_prepared": tuple(X_train_prepared.shape),
        "X_test_prepared":  tuple(X_test_prepared.shape),
        "X_train_selected": tuple(X_train_selected.shape),
        "X_test_selected":  tuple(X_test_selected.shape),
    }
}

with open("stage4_feature_pipeline.pkl", "wb") as f:
    pickle.dump(artifacts, f)

print("Saved: stage4_feature_pipeline.pkl")


# 10.3 EXPORT FEATURE NAME LISTS TO CSV (FOR AUDIT & REUSE)
# ---------------------------------------------------------
# 10.3.1 All encoded features (pre-selection)
pd.Series(final_feature_names, name="feature_name").to_csv(
    "Stage4_feature_names_after_encoding.csv", index=False
)

# 10.3.2 Final-30 selected features (post-selection)
pd.Series(selected_feature_names, name="feature_name").to_csv(
    "Stage4_selected_features_FINAL30.csv", index=False
)

# 10.3.3 Optional variants (export only if present in memory)
if "selected_feature_names_decorrelated" in globals():
    pd.Series(selected_feature_names_decorrelated, name="feature_name").to_csv(
        "Stage4_selected_features_FINAL30_decorrelated.csv", index=False
    )

if "selected_feature_names_with_buckets" in globals():
    pd.Series(selected_feature_names_with_buckets, name="feature_name").to_csv(
        "Stage4_selected_features_FINAL30_with_buckets.csv", index=False
    )

# 10.4 ENSURE TARGET VECTORS ARE 1-D SERIES (CONSISTENT CSV SHAPE)
# -----------------------------------------------------------------
if isinstance(y_train, pd.DataFrame) and y_train.shape[1] == 1:
    y_train = y_train.iloc[:, 0]
if isinstance(y_test, pd.DataFrame) and y_test.shape[1] == 1:
    y_test = y_test.iloc[:, 0]

# Ensure TRAIN/TEST matrices are DataFrames before saving
# --------------------------------------------------------
if isinstance(X_train_selected, np.ndarray):
    X_train_selected = pd.DataFrame(X_train_selected, columns=selected_feature_names)

if isinstance(X_test_selected, np.ndarray):
    X_test_selected = pd.DataFrame(X_test_selected, columns=selected_feature_names)



# 10.5 EXPORT TRAIN/TEST MATRICES (SELECTED FEATURES) AND TARGETS
# ---------------------------------------------------------------
# These are the definitive inputs for Stage 6 (Model Selection).
X_train_selected.to_csv("X_train_stage4.csv", index=False)
X_test_selected.to_csv("X_test_stage4.csv", index=False)
y_train.reset_index(drop=True).to_csv("y_train_stage4.csv", index=False)
y_test.reset_index(drop=True).to_csv("y_test_stage4.csv", index=False)

print("Saved CSVs: feature lists + X_train/X_test/y_train/y_test for Stage 6.")



Saved: stage4_feature_pipeline.pkl
Saved CSVs: feature lists + X_train/X_test/y_train/y_test for Stage 6.
